Copyright (C) 2025 Advanced Micro Devices, Inc. All rights reserved.
Portions of this notebook consist of AI-generated content.

Permission is hereby granted, free of charge, to any person obtaining a copy

of this software and associated documentation files (the "Software"), to deal

in the Software without restriction, including without limitation the rights

to use, copy, modify, merge, publish, distribute, sublicense, and/or sell

copies of the Software, and to permit persons to whom the Software is

furnished to do so, subject to the following conditions:



The above copyright notice and this permission notice shall be included in all

copies or substantial portions of the Software.



THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR

IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,

FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE

AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER

LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,

OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE

SOFTWARE.

# CV03 Object Detection

### Lab Description
This laboratory exercise introduces **YOLOv9**, a state-of-the-art one-stage object detection model.  
Unlike traditional detectors, YOLO (“You Only Look Once”) achieves high accuracy and speed by performing detection in a single forward pass, making it suitable for real-time applications.

In this hands-on lab, you will:
- **Train** YOLOv9 on a sample dataset for ~10 epochs.  
- **Evaluate** the trained model on validation/test images.  
- Record and visualize runtime, batch size, and GPU memory usage.  
- Plot training curves to verify the learning process.

### What you can expect to learn


In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 3.1 MB/s  0:00:0036m-:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.7/772.7 kB 12.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 MB 20.1 MB/s  0:00:02 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [ultralytics] [ultralytics]


In [ ]:
import os
import sys

import torch

print("Python:", sys.version.split()[0])
print("PyTorch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA device:", torch.cuda.get_device_name(0))

Use GPU:4
Python: 3.10.18
PyTorch: 2.8.0+rocm6.4
CUDA available: True
CUDA device: AMD Radeon AI PRO R9700


In [3]:
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/home/aup/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


## 2. Download COCO128 & Create Data YAML
We use a **tiny COCO subset** (COCO128) so training finishes quickly.  


In [5]:
%cd /home/aup/data/YOLOv9_data
!unzip -q Aerial.zip -d Aerial

/home/aup/data/YOLOv9_data


## 3. Training

In [6]:
# Build a YOLOv9c model from pretrained weight
model = YOLO("yolov9c.pt")

In [7]:
model.info()

YOLOv9c summary: 358 layers, 25,590,912 parameters, 0 gradients, 104.0 GFLOPs


(358, 25590912, 0, 104.02268160000003)

In [8]:
results = model.train(
    data="/home/aup/data/YOLOv9_data/Aerial/SkyFusion-YOLOv9/data.yaml", epochs=10, imgsz=640, batch=4
)

Ultralytics 8.3.205 🚀 Python-3.10.18 torch-2.8.0+rocm6.4 CUDA:0 (AMD Radeon AI PRO R9700, 32624MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/aup/data/YOLOv9_data/Aerial/SkyFusion-YOLOv9/data.yaml, degrees=0.0, deterministic=True, device=4, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov9c.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, pati

## 4. Evaluation

In [14]:
results = model(
    "/home/aup/data/YOLOv9_data/Aerial/SkyFusion-YOLOv9/test/images/3a1fe1cd4_png_jpg.rf.0b18c4bdcbdf18d62f557e6d2a1d01ca.jpg"
)


image 1/1 /home/aup/data/YOLOv9_data/Aerial/SkyFusion-YOLOv9/test/images/3a1fe1cd4_png_jpg.rf.0b18c4bdcbdf18d62f557e6d2a1d01ca.jpg: 640x640 2 ships, 45.5ms
Speed: 2.2ms preprocess, 45.5ms inference, 2.5ms postprocess per image at shape (1, 3, 640, 640)


In [18]:
import contextlib
import pathlib

from matplotlib import pyplot as plt
from PIL import Image

OUTPUT_DIR = "/home/aup/data/YOLOv9_data/results"
for i, r in enumerate(results):
    im_bgr = r.plot()
    im_rgb = im_bgr[:, :, ::-1]

    plt.figure(figsize=(8, 8))
    plt.imshow(im_rgb)
    plt.axis("off")
    plt.title(f"Prediction #{i}")
    plt.show()

    os.makedirs(OUTPUT_DIR, exist_ok=True)
    stem = f"pred_{i}"
    with contextlib.suppress(Exception):
        stem = pathlib.Path(r.path).stem
    out_file = os.path.join(OUTPUT_DIR, f"{stem}_pred.jpg")
    Image.fromarray(im_rgb).save(out_file)
    print("Saved:", out_file)

<Figure size 800x800 with 1 Axes>

Saved: /home/aup/data/YOLOv9_data/results/3a1fe1cd4_png_jpg.rf.0b18c4bdcbdf18d62f557e6d2a1d01ca_pred.jpg
